In [1]:
%reload_kedro

2020-01-05 16:48:04,648 - root - INFO - ** Kedro project sneakairs
2020-01-05 16:48:04,651 - root - INFO - Defined global variable context


In [2]:
import pandas as pd
pd.set_option('max.columns',50)
import plotly.express as px

In [3]:
df = context.io.load('KOF_04152019_05')

2020-01-05 16:48:05,406 - kedro.io.data_catalog - INFO - Loading data from `KOF_04152019_05` (CSVLocalDataSet)...


In [4]:
df['date'] = df['date'].astype('datetime64[ns]')

In [5]:
df['year'] = df['date'].dt.year

In [6]:
df['price_premium'] = round(((df['avg_resale_stockx'] - df['retail_price'])/df['retail_price'])*100,2)

2020-01-05 16:48:05,991 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [7]:
df['date'].max()

Timestamp('2019-04-15 00:00:00')

In [8]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,silhouette,price_diff,net_profit,profitable,brand_code,black,white,brown,red,blue,yellow,orange,green,purple,multi_color,main_color,main_color_id,womens,bcollab,og,sp,qs,sb,ls,nrg,prm,nsw,retro,se,pe,gs,hs,year,price_premium
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,Air Foamposite One,85,49.545275,1,6,0.864923,0.796686,0.722427,0.843831,0.758602,0.727155,0.705313,0.68003,0.700628,0.541327,black,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,38.64


## Avg net profit by silhouette (min. 2 shoes, 2018 - apr 2019)

In [353]:
num_shoes = df.groupby(['silhouette'])['code_style'].count().reset_index(name='num_shoes')

In [354]:
df = pd.merge(df, num_shoes, how='left', on='silhouette')

In [355]:
avg_net = df[df['num_shoes'] >= 2.0]

In [356]:
avg_net = avg_net[avg_net['date'] >= '2018-01-01']

In [357]:
avg_net = avg_net.groupby(['brand', 'silhouette'])['net_profit'].mean().reset_index(name='avg_net_profit')

In [358]:
avg_net = avg_net.sort_values(by=['brand','avg_net_profit'], ascending=False)

In [359]:
avg_net = avg_net[(avg_net['brand']=='Adidas') | (avg_net['brand']=='Air Jordan') | (avg_net['brand']=='Nike') | 
       (avg_net['brand']=='Reebok')]

In [360]:
avg_net = avg_net.reset_index(drop=True)

In [361]:
avg_net = pd.concat([avg_net[avg_net['brand']=='Nike'].head(5),
           avg_net[avg_net['brand']=='Air Jordan'].head(5),
           avg_net[avg_net['brand']=='Adidas'].head(5)])
           

In [362]:
avg_net=avg_net.sort_values(['brand','avg_net_profit'], ascending=[True, False])

In [366]:
fig = px.bar(avg_net, x='silhouette', y='avg_net_profit', facet_col='brand', color='brand',
            labels={'brand':'Brand', 'avg_net_profit':'Average Net Profit ($)', 'silhouette':'Sneaker Name'},
            title = 'Top 5 sneakers by average net profit for the big 3 sneaker companies<br>' + 
            '<i>Jan 2018 - Apr 2019, Minimum 2 sneakers made for each brand')
fig.update_layout(xaxis_tickangle=45, xaxis2_tickangle=45, xaxis3_tickangle=45, width=1000,
                 height=600)
fig.update_xaxes(matches=None)
fig.show()

## Avg net profit by silhouette by year (min. 5 shoes, apr 2014 - apr 2019)

In [8]:
num_shoes = df.groupby(['silhouette'])['code_style'].count().reset_index(name='num_shoes')

In [9]:
df = pd.merge(df, num_shoes, how='left', on='silhouette')

In [10]:
avg_net = df[df['num_shoes'] >= 5.0]

In [11]:
avg_net = avg_net[avg_net['date'] >= '2015-01-01']


In [12]:
avg_net = avg_net.groupby(['brand', 'silhouette', 'year'])['net_profit'].mean().reset_index(name='avg_net_profit')

In [13]:
avg_net = avg_net.sort_values(by=['brand','avg_net_profit'], ascending=False)

In [14]:
avg_net = avg_net[(avg_net['brand']=='Adidas') | (avg_net['brand']=='Air Jordan') | (avg_net['brand']=='Nike')]

In [15]:
avg_net = avg_net.reset_index(drop=True)

In [16]:
# avg_net = pd.concat([avg_net[avg_net['brand']=='Reebok'].head(5),
#            avg_net[avg_net['brand']=='Nike'].head(5),
#            avg_net[avg_net['brand']=='Air Jordan'].head(5),
#            avg_net[avg_net['brand']=='Adidas'].head(5)])
           

In [17]:
avg_net=avg_net.sort_values(['year','brand','avg_net_profit'], ascending=[False,True, False])

In [26]:
fig = px.bar(avg_net, x='silhouette', y='avg_net_profit', facet_col='brand', facet_row='year',color='brand',
            labels={'brand':'Brand', 'avg_net_profit':'Average Net Profit ($)', 'silhouette':'Sneaker Name'},
            title = 'Top 5 sneakers by average net profit for the big 3 sneaker companies<br>' + 
            '<i>2015 - 2019, Minimum 5 sneakers made for each brand')
fig.update_layout(width=1000,
                 height=2000)
# xaxis_tickangle=45, xaxis2_tickangle=45, xaxis3_tickangle=45, xaxis4_tickangle=45, 
fig.update_xaxes(matches=None)
fig.show()

### Time series

In [33]:
df5 = df[df['date'] >= '2015-01-01']

In [34]:
all_brands = df5.groupby(['brand', 'silhouette'])['code_style'].count().reset_index(name='num_shoes')

In [35]:
nike = all_brands[all_brands['brand'] == 'Nike']
adidas = all_brands[all_brands['brand'] == 'Adidas']
aj = all_brands[all_brands['brand'] == 'Air Jordan']

In [36]:
nike5 = nike.sort_values(by='num_shoes', ascending=False).head(3)
adidas5 = adidas.sort_values(by='num_shoes', ascending=False).head(3)
aj5 = aj.sort_values(by='num_shoes', ascending=False).head(3)

In [37]:
top5 = pd.concat([nike5,adidas5,aj5])
top5 = list(top5.silhouette.unique())

In [88]:
df5 = df5[df5['silhouette'].isin(top5)]

In [89]:
df5 = df5.groupby(['brand','silhouette', 'year'])['net_profit'].mean().reset_index(name='avg_net_profit')

In [93]:
df5['year'] = df5.year.astype('str')
df5 = df5.sort_values(by='year')

In [153]:
fig = px.line(df5, x='year', y='avg_net_profit', line_group='silhouette', color='brand', line_dash='silhouette',
        labels={'brand':'Brand', 'avg_net_profit':'Average Net Profit ($)', 'year':'Year'},
        title = 'Top 5 sneakers by average net profit for the big 3 sneaker companies<br>' + 
        '<i>2015 - 2019, Minimum 5 sneakers made for each brand')
fig.update_layout(width=1200,
                 height=800)
fig.show()

## Mapping sentiment vs. resale price

In [26]:
filtered = df[df['date'] >= '2015-01-01']

In [27]:
filtered = filtered[(filtered['brand']=='Adidas') | (filtered['brand']=='Air Jordan') | (filtered['brand']=='Nike')]

In [28]:
filtered = filtered.sort_values(by='brand')

In [29]:
fig = px.scatter(filtered,x='kof_wants',y='price_diff', color='brand',hover_name='name', 
           color_discrete_sequence=['#000000', '#FD0200','#FF6600'],
            title='Mapping sneaker price differential (resale - retail) vs. sentiment (Kicks on Fire "Wants") <br>'
            + '<i>Jan 2015 - Apr 2019, Big 3 companies',
            labels={'brand':'Brand', 'price_diff':'Price Difference ($)', 
                    'kof_wants':'Kicks on Fire "Wants"'})
fig.update_layout(width=1000,
                 height=600)
fig.show()

## Violin plot

In [375]:
brands_filtered = filtered[(filtered['brand']=='Adidas') | (filtered['brand']=='Air Jordan') | 
                           (filtered['brand']=='Nike')]

In [376]:
brands_filtered = brands_filtered.sort_values(by='brand')

In [382]:
fig = px.violin(brands_filtered, x='brand', y='price_diff', color='brand',
          points="all", box=True, color_discrete_sequence=['#000000', '#FD0200','#FF6600'],
            title='Distribution of sneaker price differentials (resale price - retail price)<br>'
            + '<i>Jan 2015 - Apr 2019, Big 3 companies',
            labels={'brand':'Brand', 'price_diff':'Price Difference ($)'})
fig.update_layout(width=1000,
                 height=600)
fig.show()

# Main color

In [15]:
filtered = df[df['date'] >= '2015-01-01']

In [16]:
filtered = filtered[(filtered['brand']=='Adidas') | (filtered['brand']=='Air Jordan') | 
                           (filtered['brand']=='Nike')]

In [17]:
filtered = filtered[filtered.price_premium < filtered.price_premium.quantile(.98)]

In [18]:
filtered = filtered.sort_values(by='main_color')

In [25]:
fig = px.histogram(filtered, x='price_premium', facet_row='main_color',histnorm='probability', color='main_color'
                  ,nbins=50,color_discrete_sequence=['#000000', '#0000FF','#8B4513', '#008000', '#C9C9C9',
                                                    '#FFA500', '#800080', '#FF0000', '#F8F8FF', '#FFFF00'],
            title='Distribution of price premiums by sneaker color<br>'
            + '<i>Jan 2015 - Apr 2019, Big 3 companies',
            labels={'count':'Perecentage of Sneakers', 'main_color':'Main Color', 'price_premium':'Price Premium'})
fig.update_layout(width=1200,
                 height=2000)
# xaxis_tickangle=45, xaxis2_tickangle=45, xaxis3_tickangle=45, xaxis4_tickangle=45, 
# fig.update_xaxes(matches=None)
fig.show()

## collaborations

In [38]:
nike = df[df['brand'] == 'Nike']
adidas = df[df['brand'] == 'Adidas']
aj = df[df['brand'] == 'Air Jordan']

In [39]:
ncollab = nike.groupby(['brand','silhouette', 'bcollab'])['price_diff'].mean().reset_index()
ncollab = ncollab[ncollab['silhouette'].isin(top5)]

In [40]:
acollab = adidas.groupby(['brand','silhouette', 'bcollab'])['price_diff'].mean().reset_index()
acollab = acollab[acollab['silhouette'].isin(top5)]

In [41]:
ajcollab = aj.groupby(['brand','silhouette', 'bcollab'])['price_diff'].mean().reset_index()
ajcollab = ajcollab[ajcollab['silhouette'].isin(top5)]

In [42]:
collabs = pd.concat([ncollab,acollab,ajcollab])

In [43]:
collabs['bcollab'] = collabs.bcollab.astype('str')

In [44]:
bcollab0 = collabs[collabs['bcollab']=='0']
bcollab1 = collabs[collabs['bcollab']=='1']
collabs = pd.merge(bcollab0,bcollab1, how='left', on=['brand', 'silhouette'])

In [45]:
collabs['price_diff'] = collabs['price_diff_y'] - collabs['price_diff_x']

In [46]:
collabs['neg'] = collabs['price_diff'].apply(lambda x: '1' if x < 0 else '0')

In [47]:
collabs = collabs.sort_values(by='price_diff', ascending=True)

In [48]:
fig = px.bar(collabs, x='price_diff', y='silhouette',orientation='h', color='neg',
             color_discrete_sequence=['#f08080', '#A0DB8E'],
            title='Difference in average price differential of silhouettes collaborated on versus those that were not<br>'
            + '<i>Jan 2015 - Apr 2019, 3 most popular sneakers from each of the big 3 companies',
            labels={'silhouette':'Silhouette', 'price_diff':'Average Price Difference ($)'})
fig.update_yaxes(matches=None)
fig.update_layout(width=1000,
                 height=600)
fig.show()